In [25]:
# !pip install kaggle
# !cd ~/.kaggle
# !mv ~/Downloads/kaggle.json ./
# !chmod 600 ./kaggle.json
!pip install category_encoders


     |████████████████████████████████| 82 kB 1.2 MB/s             
     |████████████████████████████████| 9.6 MB 9.0 MB/s            
     |████████████████████████████████| 233 kB 8.3 MB/s            
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.


In [62]:
!kaggle competitions download -c caltech-cs155-2022-mp1

caltech-cs155-2022-mp1.zip: Skipping, found more recently modified local copy (use --force to force download)


In [26]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import category_encoders as ce


Import and Parse Data

In [75]:
train_data = pd.read_csv('data/LOANS_TRAIN.csv')
test_data = pd.read_csv('data/LOANS_TEST.csv')

train_X = train_data.iloc[:,:-1]
train_y = train_data.iloc[:,-1]

#Remove redundant data
# TODO: decide on which data to keep and which to drop
train_data = train_data.drop(columns=['id'])

#Strip percent(%) from int_rate
train_data['int_rate'] = train_data['int_rate'].str.rstrip('%').astype(float)

#Strip percent(%) from revol_util
train_data['revol_util'] = train_data['revol_util'].str.rstrip('%').astype(float)

#TODO: Select which data should be encoded
# ce_OHE = ce.OneHotEncoder(cols=['loan_status', 'application_type', 'home_ownership', 'verification_status'])
# train_data = ce_OHE.fit_transform(train_data)

ce_BE = ce.BinaryEncoder(cols=['loan_status'])
train_data = ce_BE.fit_transform(train_data)

train_X = train_data.iloc[:,:-1]
train_y = train_data.iloc[:,-1]

#Replace subgrade with numerical value
ce_LOO = ce.LeaveOneOutEncoder(cols=['grade', 'emp_title', 'issue_d', 'addr_state', 'purpose', 'initial_list_status', 'title', 'application_type', 'home_ownership', 'verification_status', 'sub_grade', 'zip_code', 'emp_length', 'mort_acc', 'earliest_cr_line'])
train_X = ce_LOO.fit_transform(train_X, train_y)

print(train_y[:5])
print(len(train_X.keys()))

0    1
1    0
2    1
3    1
4    1
Name: loan_status_1, dtype: int64
28


In [ ]:
print(train_data.values[0])
print(train_data.values[0][5])
count = 0

correct = 0
total = 0
for datapoint in train_data.values:
    credit_score = datapoint[5]
    outcome = datapoint[-1]
    if outcome == "Fully Paid":
        ground_truth = 1
    else:
        ground_truth = 0
    if credit_score == "A" or credit_score == "B":
        prediction = 1
    else:
        prediction = 0
    if ground_truth == prediction:
        correct += 1
    total += 1
print(f"ACCURACY: {correct/total}")


